# Univariate Gaussian MLE

## Problem Statement

We have $n=100$ pieces of indepdendent and identicially distrubuted (i.i.d) data related to some measurement that is drawn from a **normal distribution**.

We will write code that computes $\mu_{MLE}$ and $\sigma_{MLE}$, which represent the mean and standard deviation computed via MLE, respectively, of the underlying normal distribution for the dataset "MLE_dataset.npy"

In [81]:
import scipy.stats
import numpy as np


def load(path):
    '''
    loads "MLE_dataset.npy" given its path into variable 'dataset'. returns 'dataset'
    '''
    dataset = np.load(path)
    return dataset
    
def MLE(dataset):
    '''
    Input:
        dataset - numpy array of shape (100,) - containing the data drawn from unknown gaussian
        
    Output:
        mu - float - MLE estimate of mu based on data
        sigma - float - MLE estimate of sigma based on data
    '''
    mu = 0
    sigma = 0
    for i in range(dataset.size):
        mu += dataset[i]
    mu = mu/dataset.size
    for i in range(dataset.size):
        sigma += np.power(dataset[i]-mu, 2)
    sigma = sigma/dataset.size
    sigma = np.power(sigma, 1/2)
    return mu, sigma
        
#load dataset. assumes data file is in same directory as code file
dataset = load("./MLE_dataset.npy")
mu_sigma = MLE(dataset)
print (mu_sigma)

(102.65639598061982, 9.743253619192298)


# Univariate Gaussian EM

## Problem statement

We will add a layer of difficulty to the problem of estimating the underlying distribution(s) of our data. Previously. we knew that all of our data came from a single underlying normal distribution.

For this problem, we have a brand new data set consisting of $n=200$ data points. These data points are drawn from **one of two unknown gaussian distributions**.

To keep the notation consistent, we will use the subscript ID $k = \{0,1\}$ to represent which gaussian a parameter/variable is referring to.

For instance, $\mu_k$ represents the mean of the gaussian with ID $k$. $\mu_0$ represents the mean of the first gaussian, and $\mu_1$ represents the mean of the second gaussian

## Variable definitions

1. $k$: id of the gaussian distributions. {0,1} 
2. $n$: number of data points. 200 in this case
3. $\mu_k$: the mean of normal distribution $k$
4. $\sigma_k$: the std of normal distribution $k$
5. $\pi_k$: the prior probability of normal distribution $k$
7. $t$: current step number
8. $z_i = k$: represents the idea that i-th data point was drawn from gaussian $k$
9. $x_i$: i-th data point

### Task Order
1. Determine the update rules for $\mu_k^{t+1}$ and $\sigma_k^{t+1}$ and write them in the cell below
2. Code the E and M step
3. Run and test your code on "EM_dataset.py"


## E - Step (8 pts)
$P(z_i = k | x_i, \theta_k^t)$ reflects the responsibility the k-th gaussian has for the i-th data point
$P(z_i = k | x_i, \theta_k^t) = \frac{P(x_i | z_i = k, \theta_k^t)*\pi_k^t}{P(x_i | \theta_k^t)}$
$= \frac{P(x_i | z_i = k, \theta_k^t)*\pi_k^t}{\sum_{k=0}^{1}P(x_i | z_i = k, \theta_k^t)*\pi_k^t}$



## M - Step (10 pts)


$$\mu_k^{t+1} = \ ??$$

$$\sigma_k^{t+1} = \ ??$$

$$\pi_k^{t+1} = \frac{\sum_{i=1}^n P(z_i = k | x_i, \theta_k^t)}{n}$$


## Formulas


$\mu_k^{t+1} = \frac{\sum_{i=1}^n P(z_i = k | x_i, \theta_k^t)*x_i}{N_k^{t}}$


$\sigma_k^{t+1}=\sqrt{\frac{\sum_{i=1}^n P(z_i = k | x_i, \theta_k^t)*(x_i-\mu_k^{k+1})^2}{N_k^{t}}}$

$N_k^{t}=\sum_{i=1}^n P(z_i = k | x_i, \theta_k^t)$

In [82]:
import scipy.stats
import numpy as np

def load(path):
    dataset = np.load(path)
    return dataset

def em(dataset, k, n_iterations):
    '''
    Input:
        dataset - np array - containing the data
        k - int - representing the number of underlying gaussian distributions
        n_iterations - int - representing number of iterations EM should run for
        
    output:
        mus - np array shape (2,) - mus[k] is the EM estimate of the mean of the kth gaussian
        sigmas - np array shape (2,) - sigmas[k] is the EM estimate of the stdev of the kth gaussian
        pi - np array shape (2,) - pis[j] is the EM estimate of the prior of the kth gaussian
    '''
    n_samples = dataset.shape[0]

    # Initial guesses for the parameters DO NOT CHANGE
    FINAL_INITIAL_MUS = np.asarray([90.0, 210.0]) #DO NOT CHANGE
    FINAL_INITIAL_SIGMAS = np.asarray([28.0,19.0]) #DO NOT CHANGE
    FINAL_INITIAL_PIS = np.asarray([0.3,.7]) #DO NOT CHANGE
    pis = FINAL_INITIAL_PIS #DO NOT CHANGE
    mus = FINAL_INITIAL_MUS #DO NOT CHANGE
    sigmas = FINAL_INITIAL_SIGMAS #DO NOT CHANGE
    
    for em_iter in (range(n_iterations)):
            
            
            
            #E Step
            #data
            #store Pr(zi|xi, data) for each iteration
            #commputed in e step, used in m step
            d= [[0.0]*k for i in range(n_samples)]
            #Pr(xi|zi, data) store
            #used in e step
            c=[[0.0] for i in range(k)]
            #denominator of pr(zi|xi, data)
            #used in e step
            denom=0
            #store Pr(zi|xi, data)*xi
            
            #calculating Pr(zi|xi, data)
            for i in range(n_samples):
                #setting denom to 0 for the next iteration
                denom=0
                #calculating Pr(xi|zi, data)
                for a in range(k):
                    c[a]=scipy.stats.norm(mus[a], sigmas[a]).pdf(dataset[i]) 
                #adding all Pr(xi|zi, data)*pis for denom and 
                #setting numerator 
                for a in range(k):
                    d[i][a]=c[a]*pis[a]
                    denom += d[i][a]
                #dividing pr(xi|zi, data)*pis by summation of 
                #pr(xi|zi, data)*pis   
                for a in range(k):
                    d[i][a]=d[i][a]/denom

            #M step
            #only m step data
            #commputed in m step, used in m step
            x= [[0.0]*k for i in range(n_samples)]
            #store ni: the approximation of the number of 
            #rvs for each distribution 
            #used to find mus and sigmas every iteration
            #used in m step and commputed in it
            size=[[0.0] for i in range(k)]
            
            #calculating the expected data for dist. 1 and 2
            #summation Pr(zi|xi, data)*xi
            #nummerator of mus
            for i in range(n_samples):
                #iterating over each distribution
                for a in range(k):
                    x[i][a]=dataset[i]*d[i][a]
            
            #setting pis to 0 so we can find the next 
            #rounds approximations
            for i in range(k):
                pis[i]=0

            #adding all pr(zi|xi, data) for each pis
            for i in range(n_samples):
                #iterating over #distributions
                for a in range(k):
                    pis[a] += d[i][a]
                
            #dividing by n
            for i in range(k):
                pis[i] = pis[i]/n_samples
            #calculating the approximated number of samples 
            #from each distribution
            #because pis now has the current data multiplyig 
            #by n_samples now leaves us 
            #the summation of all Pr(zi|xi, data)
            for i in range(k):
                size[i]=n_samples*pis[i]
            #setting mus to zero to caculate next rounds
            for i in range(k):
                mus[i]=0
            #adding Pr(zi|xi, data)*xi for all samples
            for i in range(n_samples):
                #itertating over each distribution
                for a in range(k):
                    mus[a] += x[i][a]
            #dividing summation Pr(zi|xi, data)*xi 
            #by the aprroximated sizes
            for i in range(k):
                mus[i]=mus[i]/size[i]
            #setting sigmas to 0 for new calculation
            for i in range(k):
                sigmas[i]=0
            #adding Pr(zi|xi, data)*(xi-mus)^2 for each sigmas
            for i in range(n_samples):
                #iterating over each distribution
                for a in range(k):
                    sigmas[a] +=d[i][a]* np.power((dataset[i]-mus[a]), 2)
            #dividing by approximated size and taking 
            #square root to find sigmas instead of sigmas^2
            for i in range(k):
                sigmas[i] = sigmas[i]/size[i]
                sigmas[i] = np.power(sigmas[i], 1/2)
            
            
    return pis, mus, sigmas

def main():
    n_iterations = 20 #DO NOT CHANGE
    k = 2 #DO NOT CHANGE
    #load dataset. assumes data file is in 
    #same directory as code file
    dataset = np.load("EM_dataset.npy")
    pis, mus, sigmas = em(dataset, k, n_iterations)
    print("Pis:")
    print(pis)
    print("Mus:")
    print(mus)
    print("Sigmas:")
    print(sigmas)
main()

Pis:
[0.41534395 0.58465605]
Mus:
[100.50444169 201.90594025]
Sigmas:
[23.10763968 19.8353594 ]
